# Trabalho de Bioinformática
- Ana Cristina Silva de Oliveira, 11965630
- Fernando Henrique Paes Generich, 11795342
- Vítor Amorim Fróis, 12543440

In [1]:
import pandas as pd
from Bio import Entrez, SeqIO, Seq
from tqdm import tqdm
import matplotlib.pyplot as plt

## Leitura dos arquivos .gff3
Vamos selecionar todas as linhas em que a Strand é positiva:

In [ ]:
filenames = [
  'TEAnnotationFinal_Helitron.gff3',
  'TEAnnotationFinal_LINE.gff3',
  'TEAnnotationFinal_LTR.gff3',
  'TEAnnotationFinal_MITE.gff3',
  'TEAnnotationFinal_SINE.gff3',
  'TEAnnotationFinal_TIR.gff3'
]

def process_file(filename: str) -> pd.DataFrame:
  df = pd.read_csv(f'data/{filename}', sep='\t', header=None)
  df.drop(df[ df[6] != '+' ].index , inplace = True)
  df = df.drop( [1, 5, 7, 8], axis=1)
  return df

te_df = pd.DataFrame()

# Juntando os dados em um único DataFrame
for f in filenames:
  te_df = pd.concat([te_df, process_file(f)], ignore_index=True)

# Renomeando colunas
te_df.rename(columns={0 : "Chr", 2: "Class", 3: "Start", 4: "End", 6: "Strand"}, inplace = True)

# Seleciona Chr numérico apenas
te_df = te_df[te_df['Chr'].astype(str).str.isdigit()]

te_df.head()

## Obtendo as sequências de cromossomos do NCBI
No total, são 10 cromossomos

In [ ]:
Entrez.email = "fernando_gene@usp.br"

ind = 618874

allchromosomes = []

for i in tqdm(range(10)):
  gen_bank_term = "LR" + str(ind+i) + ".1"

  handle = Entrez.esearch(db="nucleotide", term=gen_bank_term, retmax="10")
  rec_list = Entrez.read(handle)
  handle.close()

  id_list = rec_list['IdList']
  handle = Entrez.efetch(db='nucleotide', id=id_list, rettype='fasta', retmode="text")
  recs = list(SeqIO.parse(handle, 'fasta'))
  handle.close()

  allchromosomes.append(recs[0])

In [ ]:
chromosome_dict = {
  'Number': [],
  'Sequence': []
}

for i in allchromosomes:
  chromosome_dict['Number'].append(i.id)
  chromosome_dict['Sequence'].append(i.seq)

chromosome_df = pd.DataFrame(chromosome_dict)

chromosome_df

## Efetuando um Join entre os DataFrames 

In [ ]:
def get_sequence(chromosome: int, start: int, end: int) -> Seq.Seq:
  return chromosome_df.Sequence[chromosome-1][start:end+1]

te_df['Sequence'] = te_df.apply(lambda x: get_sequence(int(x.Chr), int(x.Start), int(x.End)), axis=1)

Deleta linhas que possuem NaN

In [ ]:
te_df = te_df.dropna()
te_df.head()

Salva o Dataframe como `.csv`

In [ ]:
te_df.to_csv('data/transposable_elements.csv')

Leitura do arquivo csv

In [2]:
te_df = pd.read_csv('data/tranposable_elements.csv', index_col=0)

In [3]:
te_df.head()

,Chr,Class,Start,End,Strand,Sequence
0,7,Class II subclass 2/Helitron/Helitron,82856122,82857584,+,AGCTTCGTCACCAGCTTTGCTCCGACCACCCTTTGTCCATACTAAC...
1,1,Class II subclass 2/Helitron/Helitron,239302471,239302834,+,TCAGGGTTGCTTCTTGGCGAAGACAGGGCCTCGGGCGAGCCAGAAA...
2,8,Class II subclass 2/Helitron/Helitron,2261318,2261604,+,CGCCCAAGCAGACGGTCACCATCAGCGAAGACCTCACTTCGCATGA...
3,5,Class II subclass 2/Helitron/Helitron,124665404,124665608,+,ATGCCAAGTCGTGTCAAACGACTTAGGGTAGGGGTCAACTTTCTCC...
4,9,Class II subclass 2/Helitron/Helitron,77998293,78002783,+,TTAGGTTATTTATATACTAGTTTATGTTGATGATATAATCATCACT...


## Classificação com Kernels

In [4]:
from Levenshtein import distance as levenshtein_distance
import numpy as np
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
small_df = te_df.dropna().sample(1000).reset_index(drop=True)

In [6]:
def edit_distance_kernel(A, B):
    kernel_matrix = np.zeros((len(A), len(B)))
    for a in A:
        for b in B:
            i = int(a[0])
            j = int(b[0])
            kernel_matrix[i, j] = levenshtein_distance(
                small_df.iloc[i]['Sequence'], 
                small_df.iloc[j]['Sequence']
            )
    return np.exp(-kernel_matrix/normalizing_term)

### Split entre treino e teste

In [15]:
X = list(small_df['Sequence'])
y = np.array(small_df['Chr']).reshape(-1, 1)

le = LabelEncoder()
y = le.fit_transform(y)

/home/vitor/Documents/TransposableElements/venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Criação da matriz kernel

In [55]:
%%time
from strkernel.mismatch_kernel import MismatchKernel
from strkernel.gappy_kernel import gappypair_kernel as GappyPairKernel
from strkernel.mismatch_kernel import preprocess
from sklearn.kernel_approximation import Nystroem

kernel = MismatchKernel(l=4, k=5, m=1).get_kernel(preprocess(X)).kernel
# kernel = GappyPairKernel(X, k=1,t=0,g=1)

CPU times: user 26.3 s, sys: 63.5 ms, total: 26.4 s
Wall time: 26.8 s


In [60]:
exponential_kernel = np.exp(kernel)
diagonal = np.diag(1./np.sqrt(np.diag(ek)))
normalized_exponential_kernel = D @ ek @ D

### Split Treino Teste
Aqui podemos escolher entre as matrizes `kernel`, `exponential_kernel` e `normalized_exponential_kernel`

In [163]:
K_train, K_test, y_train, y_test = train_test_split(normalized_exponential_kernel, y, shuffle=False)
K_train = K_train[:, :K_train.shape[0]]
K_test = K_test[:, :K_train.shape[0]]
print(K_train.shape)
print(K_test.shape)

(750, 750)
(250, 750)


### Treinamento

In [167]:
classifier = SVC(
    kernel='precomputed', 
    verbose=False, 
    shrinking=False,
    C=1,
    class_weight='balanced',
    decision_function_shape='ovr',
    probability=True
)
classifier.fit(K_train, y_train)

SVC(C=1, class_weight='balanced', kernel='precomputed', probability=True,
    shrinking=False)

### Avaliação do modelo no conjunto de treinamento

In [168]:
y_pred = classifier.predict(K_train)
accuracy_score(y_pred, y_train)

0.6906666666666667

### Avaliação do modelo no conjunto de teste

In [169]:
y_pred = classifier.predict(K_test)
accuracy_score(y_pred, y_test)

0.116